## 1. 필요한 라이브러리 임포트

In [3]:
%pip install pandas numpy matplotlib seaborn openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns        

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## 2. 한국의 미세먼지 데이터 로드 및 전처리

In [ ]:
import pandas as pd

# 1. 엑셀 파일 불러오기
file_path = "./data/2022년 대기오염물질 배출량 통계(시군구별 배출원소분류별 연료별).xlsx"
df_raw = pd.read_excel(file_path, sheet_name='2022년')

# 2. 상위 3행 제거 (메타데이터 + 헤더행)
df = df_raw.iloc[3:].reset_index(drop=True)

# 3. 컬럼명 지정 (1번째, 2번째 행을 조합)
columns_lvl_1 = df_raw.iloc[1].tolist()
columns_lvl_2 = df_raw.iloc[2].tolist()
final_columns = [a if pd.notna(a) else b for a, b in zip(columns_lvl_1, columns_lvl_2)]
df.columns = final_columns

# 4. 필요한 열만 선택
df_filtered = df[["시도", "배출원대분류", "PM-2.5", "PM-10"]].copy()
df_filtered["PM-2.5"] = pd.to_numeric(df_filtered["PM-2.5"], errors="coerce")
df_filtered["PM-10"] = pd.to_numeric(df_filtered["PM-10"], errors="coerce")
df_filtered = df_filtered.dropna(subset=["시도", "배출원대분류", "PM-2.5", "PM-10"])
df_filtered = df_filtered[df_filtered["시도"] != "바다"]

# 5. 시도 + 배출원대분류 기준으로 총합 계산
summary_pm25 = df_filtered.groupby(["시도", "배출원대분류"])["PM-2.5"].sum().reset_index()
summary_pm10 = df_filtered.groupby(["시도", "배출원대분류"])["PM-10"].sum().reset_index()

# 6. 피벗 테이블 생성
pivot_pm25 = summary_pm25.pivot(index="시도", columns="배출원대분류", values="PM-2.5").fillna(0).astype(int)
pivot_pm10 = summary_pm10.pivot(index="시도", columns="배출원대분류", values="PM-10").fillna(0).astype(int)

# 6-1. '총합' 열 추가
pivot_pm25["총합"] = pivot_pm25.sum(axis=1)
pivot_pm10["총합"] = pivot_pm10.sum(axis=1)

# 6-2. 인덱스 리셋 후 '시도' 축약
pivot_pm25 = pivot_pm25.reset_index()
pivot_pm10 = pivot_pm10.reset_index()

# 6-3. 시도 축약 매핑
name_map = {
    "서울특별시": "서울",
    "부산광역시": "부산",
    "대구광역시": "대구",
    "인천광역시": "인천",
    "광주광역시": "광주",
    "대전광역시": "대전",
    "울산광역시": "울산",
    "세종특별자치시": "세종",
    "경기도": "경기",
    "강원도": "강원",
    "충청북도": "충북",
    "충청남도": "충남",
    "전라북도": "전북",
    "전라남도": "전남",
    "경상북도": "경북",
    "경상남도": "경남",
    "제주특별자치도": "제주"
}

pivot_pm25["시도"] = pivot_pm25["시도"].map(name_map)
pivot_pm10["시도"] = pivot_pm10["시도"].map(name_map)

# 6-4. "세종" 제거
# pivot_pm25 = pivot_pm25[pivot_pm25["시도"] != "세종"]
# pivot_pm10 = pivot_pm10[pivot_pm10["시도"] != "세종"]

# 8. 확인용 출력
print("=== PM-2.5 피벗 테이블 ===")
print(pivot_pm25.head())

print("\n=== PM-10 피벗 테이블 ===")
print(pivot_pm10.head())


> **같은 배출량이더라도, 오염원의 배출 위치, 방식, 화학 성분, 기상 조건에 따라 대기질 영향은 달라진다**는 것은  
> 환경과학에서 인정된 객관적 사실입니다.

따라서, 단순히 `t/yr` 배출량만으로 대기질 기여도를 판단하기보다는,  
**질량 기준 + 배출 특성 + 기상 조건**을 함께 고려해야 합니다.

| 오염원   | PM2.5 배출량 | 배출 조건             | 대기질 영향              |
|----------|---------------|------------------------|---------------------------|
| 발전소   | 100kg         | 고고도, 확산 양호       | 낮음                      |
| 차량     | 100kg         | 지표면, 교통 밀집        | 매우 높음                 |
| 가정 난방 | 100kg         | 겨울철 밤, 지면 근처     | 높음                      |
| 농업 연소 | 100kg         | 야외, 바람 강함          | 지역에 따라 낮을 수도 있음 |


3. 날씨 요인 데이터 로드 및 전처리

3. 중국의 미세먼지 데이터 로드 및 전처리

5. 황사 여부 

6. 다중공산성 확인 

7. 회귀분석 모델 작성

8. 프로젝트 시각화